In [1]:
# Let's load necessary libraries and the datasets
%load_ext autoreload
import pandas as pd
#using networkx for triads
import networkx as nx
PATH = "data/"
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import combinations
import csv
import pickle 

%autoreload 2

#Reloading the parsed datasets + triad dicts + table1 in case
#Epinions
epinions = pd.read_csv(PATH+"soc-sign-epinions.txt", sep="\t", 
                       header=None, names = ["FromNodeId", "ToNodeId", "Sign"], skiprows = 4)
epinions.drop(index=epinions.query('FromNodeId == ToNodeId').index, inplace=True)
epinions.reset_index(inplace=True,drop=True)

#Slashdot
slashdot = pd.read_csv(PATH+"soc-sign-Slashdot090221.txt",sep= "\t", 
                       header=None, names = ["FromNodeId", "ToNodeId", "Sign"], skiprows = 4)
slashdot.drop(index=slashdot.query('FromNodeId == ToNodeId').index, inplace=True)
slashdot.reset_index(inplace=True,drop=True)

#Wiki
wiki = pd.read_csv(PATH+"parsed_wiki.csv", header=0)
wiki.drop(index=wiki.query('FromNodeId == ToNodeId').index, inplace=True)
wiki.reset_index(inplace=True,drop=True)

#Table 1
with open('./data/table1.pkl', 'rb') as f:
    table1 = pd.read_pickle(f)
pd.options.display.float_format = '{:,.3f}'.format
table1.loc[["Nodes","Edges","Triads"]] = table1.loc[["Nodes","Edges","Triads"]].astype(int)
table1

,Epinions,Slashdot,Wikipedia
Nodes,131828,82140,7118
Edges,841372,549202,103747
+edge,85.300,77.400,78.800
-edge,14.700,22.600,21.200
Triads,13317672,1508105,790532


This part of the replication is largely built NetworkX. The proposed functions didn't work, so I figured I'd go into the source code and adapt it to my use. Basically, I wanted to get the types of triad for a given set of 3 nodes, then decide how to count them.

Also, I have taken a piece of code from this [link](https://www.cl.cam.ac.uk/teaching/1314/L109/tutorial.pdf), just the loop from page 41, to loop on all nodes efficiently because initially, I used a directed view of the graph only to generate all triads and loop (see hidden Raw cell below if you want to check that), but after having the method run for over 4 hours on Wikipedia alone, I gave up and looked for another way.

In my final version of `get_triad_types`, I used an undirected view of the graph to get neighbors more easily (see docstring), then used a directed view of the graph to get all the triad infos I needed.

**For a given triplet of connected nodes, I get the tricode (from NetworkX sourcecode) and get its name to see whether it was a triad of interest.**

### COMMENTS FOR THE ASSISTANTS HERE (the rest is just my textual description for the final submission)

So. This is what I do, in bulletpoints.

- Algorithm : 
    - Instead of triadic census, I use a for iteration on nodes and its neighbours to get the triangles. 
    - Using sourcecode from NetworkX, I get the triangle types and only keep the ones of interest 
    - Then, from those triangles, I get the edges and form combinations of 3 edges.
    - Then I find the closed loops to keep valid triangles. 
    - **From those closed triangles, I get the edge signs, and simply add them (see `sum_to_type` below)** (this is the part where I classify a triangle to its type, T1, T2, etc.)
    - Then I simply add the count +=1 (since the count is correct and the classification is not, I don't know)
- After this, I shuffle the edge signs randomly using `pd.df.sample(frac=1)` (see `shuffle_sign` below) and repeat my algorithm and save all the results in table3 (see `get_results_table3`)

In [2]:
#==================My own method below=====================
def sum_to_type(number):
    """
    Converts the sum of signs (+/- 1) in a given triangle to its type of triads 
    returns a string.
    """
    type_name = ''
    if number == 3:
        type_name = 'T3+++'
        
    if number == 1:
        type_name = 'T2++-'
        
    if number == -1:
        type_name = 'T1+--'
        
    if number == -3:
        type_name = 'T0---'
    return type_name
#==========================================================

#utilities&methods that are within the NetworkX package source code, on which I will build my method for triad 
def _tricode(G, v, u, w):
    """Returns the integer code of the given triad.

    This is some fancy magic that comes from Batagelj and Mrvar's paper. It
    treats each edge joining a pair of `v`, `u`, and `w` as a bit in
    the binary representation of an integer.

    """
    combos = ((v, u, 1), (u, v, 2), (v, w, 4), (w, v, 8), (u, w, 16), (w, u, 32))
    return sum(x for u, v, x in combos if v in G[u])

#: The integer codes representing each type of triad.
#: Triads that are the same up to symmetry have the same code.
TRICODES = (1, 2, 2, 3, 2, 4, 6, 8, 2, 6, 5, 7, 3, 8, 7, 11, 2, 6, 4, 8, 5, 9,
            9, 13, 6, 10, 9, 14, 7, 14, 12, 15, 2, 5, 6, 7, 6, 9, 10, 14, 4, 9,
            9, 12, 8, 13, 14, 15, 3, 7, 8, 11, 7, 12, 14, 15, 8, 14, 13, 15,
            11, 15, 15, 16)

#: The names of each type of triad. The order of the elements is
#: important: it corresponds to the tricodes given in :data:`TRICODES`.
TRIAD_NAMES = ('003', '012', '102', '021D', '021U', '021C', '111D', '111U',
               '030T', '030C', '201', '120D', '120U', '120C', '210', '300')

#: A dictionary mapping triad code to triad name.
TRICODE_TO_NAME = {i: TRIAD_NAMES[code - 1] for i, code in enumerate(TRICODES)}

#Triad names of interest (see replication_report1)
names_interest_single = ["030T","030C"]
names_interest_multiple = ["120D","120U","120C","210", "300"]

If the three nodes form a triplet of interest, I check which type it is. 
Then I generate, for all the edges associated with the subgraph of those three nodes, a combination of 3 edges, to form all possible triplets.

From those, I must filter those that do not form a closed loop, i.e. I only keep the triplets where every node is accessible from any node.
Then, I keep those triangles, get the sign attributes sum, and get which type each triangle is (T3, T2, T1, T0) using `sum_to_type()` defined above


In [3]:
def count_triad_types(pandas_edgelist):
    """
    Built upon some items from the source code of networkX such as _tricode, 
    tricode_to_name dict etc as well a loop of code I've found from 
    https://www.cl.cam.ac.uk/teaching/1314/L109/tutorial.pdf
    which provides an iteration which is infinitely faster than the whatever is coded by NetworkX's creators
    
    Input : pandas_edgelist to create Directed AND undirected version of the same graph
    Output : a pandas.DF of the count of each type of triads
    
    for a given NetworkX directed Graph G
    A triangle is formed from 3 connected nodes.
    A triangle is a valid one if it forms a loop.
    We verify that the 3 nodes form a triad of interest by checking against 
    Triad name using tricode_to_name
    ex : 
        ((6,3), (6,10), (10,3)) is a valid triangle.
        ((6,3), (6,10), (3,6)) is not valid.
        
    Below, for tri in triangles, tri contains the edges and their sign.
    ex: for ((6,3), 1)
        tri[x][0] gives the tuple corresponding to the x'th edge in a triangle
        tri[x][1] gives the sign of the x'th edge in a triangle
    
    Then, getting sum of signs of valid triangles(using mask), I count the value
    of signs for each triangle (3 = +++, 1 = ++-, -1 = +--, -3 = ---)
    """
    print("Creating graphs")
    G_directed = nx.from_pandas_edgelist(pandas_edgelist, "FromNodeId","ToNodeId",edge_attr="Sign",create_using=nx.DiGraph)
    undirected = nx.from_pandas_edgelist(pandas_edgelist, "FromNodeId","ToNodeId",edge_attr="Sign",create_using=nx.Graph)
    
    result = pd.DataFrame(np.zeros((4,2),dtype=np.int64),
                          index=['T3+++','T1+--','T2++-','T0---'],
                          columns = ['count','fraction'])
    # iteration over all nodes
    print("Iterating")
    nodes = undirected.nodes()
    #Get neighbors. Use undirected view to get to/from nodes at once
    #This same code for neighbors using a directed view yielded only nodes to which n0 was connected,
    #rather than both TO_nodes and FROM_nodes for n0, n1, etc.
    for n0 in nodes:
        neighbors1 = set(undirected[n0]) 
        for n1 in filter(lambda x: x>n0, neighbors1):
            neighbors2 = set(undirected[n1])
            common = neighbors1 & neighbors2 #common neighbors
            
            #Filter avoids repetition
            for n2 in filter(lambda x: x>n1, common): 
            
                #From this part on, I use directed graph for triad analysis.
                name = TRICODE_TO_NAME[_tricode(G_directed, n0, n1, n2)]
                
                #Early continue if name is not of interest, avoids computing billions of triads
                if name not in (names_interest_single+names_interest_multiple):
                    continue
                    
                #Get edges of the subgraph of 3 nodes (triangle), then check which type 
                #of triad it is and do appropriate calculations
                edgeview = G_directed.subgraph((n0,n1,n2)).edges()
                
                if name in names_interest_single:
                    sign_sum = np.array([(edge, edgeview[edge]['Sign']) for edge in edgeview],
                                        dtype=object).sum(axis=0)[1]
                
                    result.loc[sum_to_type(sign_sum)]['count'] += 1
                    
                elif name in names_interest_multiple:
        
                    #Get the set of edges and their sign for this given triplet of interest
                    list_edges = [(edge, edgeview[edge]['Sign']) for edge in edgeview]
        
                    #Form all the triangles possible (combinations of 3 edges)
                    triangles = list(combinations(list_edges,3))
                    
                    #Get mask to get all valid triangles, all true by default
                    valid_loops_index = np.ones(len(triangles), dtype = bool)
                    
                    for idx, tri in enumerate(triangles):
                        loop = tuple((tri[0][0], tri[1][0], tri[2][0]))
                        # Counting each time a node appears in a given loop.
                        # Valid loop must have each node appear 2 times.
                        _, count = np.unique(loop, return_counts=True)
                        if (1 in count) or (3 in count):
                            #Set invalid to false
                            valid_loops_index[idx] = False
                    #Using fancy indexing, taking the sum of axis = 1 so we get the sum of sign with [:,1] slicing
        
                    signs_sum = np.array(triangles,dtype=object)[valid_loops_index].sum(axis=1)[:,1]
                    
                    for number in np.unique(signs_sum):
                        result.loc[sum_to_type(number)]['count'] += len(signs_sum[signs_sum==number])
    
    result['fraction'] = result['count']/ result['count'].sum()
    print("DONE :: Returning dataframe containing results")
    return result

In [4]:
#To get P0, we must shuffle the edge signs, then re-run the functions to get the fractions.
def shuffle_sign(df):
    """
    Uses pd.df.sample(frac=1) to shuffle all rows, i.e. shuffle while keeping the distribution.
    By dropping before shuffling and re-merging after, we shuffle only the signs.
    """
    df_return = df.copy()
    sign_shuffled = df.drop(columns=['FromNodeId','ToNodeId'])
    sign_shuffled = sign_shuffled.sample(frac=1)
    sign_shuffled.reset_index(inplace=True, drop=True)
    df_return['Sign'] = sign_shuffled['Sign']
    return df_return

In [5]:
#Giving count_triad_types the pandas_edgelists (shuffled and unshuffled and printing results)

#Shuffling DFs
epinions_shuffled = shuffle_sign(epinions)
slashdot_shuffled = shuffle_sign(slashdot)
wiki_shuffled = shuffle_sign(wiki)

#Getting the triad counts for normal and shuffled
epi_triads = count_triad_types(epinions)
epi_triads_shuffled = count_triad_types(epinions_shuffled)

slashdot_triads = count_triad_types(slashdot)
slashdot_triads_shuffled = count_triad_types(slashdot_shuffled)

wiki_triads = count_triad_types(wiki)
wiki_triads_shuffled = count_triad_types(wiki_shuffled)

Creating graphs
Iterating
DONE :: Returning dataframe containing results
Creating graphs
Iterating
DONE :: Returning dataframe containing results
Creating graphs
Iterating
DONE :: Returning dataframe containing results
Creating graphs
Iterating
DONE :: Returning dataframe containing results
Creating graphs
Iterating
DONE :: Returning dataframe containing results
Creating graphs
Iterating
DONE :: Returning dataframe containing results


In [14]:
def get_results_table3(df_normal, df_shuffled):
    """
    Get the results formatted like table3 from the result dataframes returned by count_triad_types
    Requires both tables returned using both the normal and shuffled signs dfs
    """
    table3 = pd.DataFrame(np.zeros((4,4),dtype=np.int64),
                               index=['T3+++','T1+--','T2++-','T0---'],
                               columns = ['count','fraction','prob_p0','surprise'])
    table3['count'] = df_normal['count']
    table3['fraction'] = df_normal['fraction']
    table3['prob_p0'] = df_shuffled['fraction']
    
    expected = table3['prob_p0']*table3['count'].sum()
    denominator = np.sqrt(expected*(1-table3['prob_p0']))
    table3['surprise'] = (table3['count']-expected)/denominator
    return table3

In [23]:
#Initializing Table3 to get the results
table3_epinions = get_results_table3(epi_triads, epi_triads_shuffled)

table3_slashdot = get_results_table3(slashdot_triads, slashdot_triads_shuffled)

table3_wiki = get_results_table3(wiki_triads, wiki_triads_shuffled) 


In [22]:
#Displaying with formatting
print("Table 3 : \n")
print("#========= Epinions =========#")
display(table3_epinions.style.format({'fraction': "{:,.3f}", 'prob_p0': '{:,.3f}', 'surprise' : '{:,.1f}'}))
print("#========= Slashdot =========#")
display(table3_slashdot.style.format({'fraction': "{:,.3f}", 'prob_p0': '{:,.3f}', 'surprise' : '{:,.1f}'}))
print("#========= Wikipedia =========#")
display(table3_wiki.style.format({'fraction': "{:,.3f}", 'prob_p0': '{:,.3f}', 'surprise' : '{:,.1f}'}))

Table 3 : 

#========= Epinions =========#


,count,fraction,prob_p0,surprise
T3+++,11616708,0.872,0.621,"1,889.2"
T1+--,688557,0.052,0.055,-54.2
T2++-,924739,0.069,0.321,"-1,963.9"
T0---,87668,0.007,0.003,223.3


#========= Slashdot =========#


,count,fraction,prob_p0,surprise
T3+++,1266646,0.840,0.464,924.6
T1+--,115884,0.077,0.119,-158.6
T2++-,109303,0.072,0.406,-833.5
T0---,16272,0.011,0.011,-5.6


#========= Wikipedia =========#


,count,fraction,prob_p0,surprise
T3+++,555300,0.702,0.494,371.5
T1+--,63425,0.080,0.104,-69.7
T2++-,163328,0.207,0.393,-339.2
T0---,8479,0.011,0.009,12.9


Comparing with table 3 from the paper, it seems that the values for the first two columns for T2++- and T1+-- have been reversed. Indeed, looking at Wikipedia for example, we see that the count for T1+-- is 63 425 instead of 163328 like in the paper. As a result, the fraction $p(T_{i})$ has also been switched. 

On the other hand, we notice that the values for $p_{0}(T_{i})$ are the same as in table3. This is quite odd, as the same method was used to compute the type of each triangle generated by the sets of 3 edges. 

I do not know whether I have made a mistake, or whether the authors of the paper have. If I made a mistake, I do not know where it was from, as I believe my reasoning for the type computation seems sound. I take the sum of the signs, and simply summing $\pm1$ yields me the four different types of triads. On top of that, I use the exact same method to compute the type of triads for both the normal and the shuffled edge signs, so if I had made a mistake somewhere, the results for both DFs  would have been switched, rather than only one of the two columns.

As a result, the values of the last column (surprise) are also different as they rely on the others columns to be computed.